In [1]:
from delta import DeltaTable

ModuleNotFoundError: No module named 'delta'

In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("test_moto")
                             .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0")
                             .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
                             .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
                             .config("spark.sql.repl.eagerEval.enabled", "true")
                             .getOrCreate())

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d1422599-a222-450d-9c88-b4f85975fb03;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.1.0/delta-core_2.12-2.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.1.0!delta-core_2.12.jar (1083ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.1.0/delta-storage-2.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.1.0!delta-storage.jar (257ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.8/antlr4-runtime-4.8.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.8!antlr4-runtime.jar (272ms)
downl

In [3]:
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
hadoop_conf.set("fs.s3a.access.key", "mock")
hadoop_conf.set("fs.s3a.secret.key", "mock")
hadoop_conf.set("fs.s3a.endpoint", "http://localhost:5000")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")

In [3]:
df = spark.read.csv('/workspace/dados_limpos_saude.csv', header=True)

In [4]:
from pyspark.sql import functions as F
from pyspark.sql import Window
window_spec = Window.partitionBy('Cargo').orderBy(F.col('Objetiva').desc(), F.col('Nascimento'))

df2 = df.select('Nome', 
          'Cargo', 
          F.col('Inscrição').cast('int').alias('Inscrição'),
          F.to_date('Nascimento', 'dd/MM/yyyy').alias('Nascimento'),
          F.col('Objetiva').cast('float').alias('Objetiva')
          ).withColumn('Posição', F.row_number().over(window_spec)).orderBy('Nome')
df2.printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Inscrição: integer (nullable = true)
 |-- Nascimento: date (nullable = true)
 |-- Objetiva: float (nullable = true)
 |-- Posição: integer (nullable = false)



In [5]:
df2

Nome,Cargo,Inscrição,Nascimento,Objetiva,Posição
ABNER GOMES DE AR...,Agente Administra...,282033,1989-08-12,68.0,208
ACSA ALICE MARTIN...,Médico PSF II,282319,1997-05-16,70.0,101
ADA KISLEY LIMA C...,Agente Administra...,265597,1981-07-04,52.0,442
ADALBERTO DAMIAO ...,Médico PSF II,284633,1988-04-06,66.0,118
ADDAN OLIVEIRA DE...,Cirurgião Dentist...,255257,1997-05-19,88.0,24
ADDSON SADY COELHO,Cirurgião Dentist...,268648,1970-10-11,64.0,16
ADELIA MARIA SILV...,Técnico de Serviç...,260454,1982-07-05,56.0,148
ADELSON XAVIER PINTO,Técnico de Serviç...,255176,1983-05-09,75.0,32
ADELUVIA COURA DO...,Enfermeiro I,260342,1982-11-12,54.0,269
ADERNES DA SILVA ...,Enfermeiro III,279569,1968-12-23,60.0,254


In [23]:
df2.filter('Cargo = "PROFESSOR DE EDUCAÇÃO - PI" and Objetiva = 52.0').orderBy(F.col('Posição')).show()

+--------------------+--------------------+---------+----------+--------+-------+
|                Nome|               Cargo|Inscrição|Nascimento|Objetiva|Posição|
+--------------------+--------------------+---------+----------+--------+-------+
|SILVANA RIBEIRO D...|PROFESSOR DE EDUC...|   267581|1970-01-01|    52.0|    491|
|ANA PAULA ESTEVES...|PROFESSOR DE EDUC...|   252959|1979-01-01|    52.0|    492|
|ANA CAROLINA PEREIRA|PROFESSOR DE EDUC...|   252317|1992-04-01|    52.0|    493|
|LUCIANA DA CONCEI...|PROFESSOR DE EDUC...|   276776|1981-07-01|    52.0|    494|
|KARLA CRISTINA DO...|PROFESSOR DE EDUC...|   268450|1982-04-02|    52.0|    495|
|ROSÁLIA BELINTANI...|PROFESSOR DE EDUC...|   254015|1968-03-03|    52.0|    496|
|MARCELA PRISCILLA...|PROFESSOR DE EDUC...|   266981|1989-05-03|    52.0|    497|
|ISABELA DE SOUZA ...|PROFESSOR DE EDUC...|   274951|1999-06-03|    52.0|    498|
|PATRICIA JARDIM D...|PROFESSOR DE EDUC...|   274046|1974-08-03|    52.0|    499|
|APARECIDA DOS S

In [7]:
df2.filter('Nome like "NATÁLIA COELHO%"').show(truncate=False)

+----------------------+------------------------------------+---------+----------+--------+-------+
|Nome                  |Cargo                               |Inscrição|Nascimento|Objetiva|Posição|
+----------------------+------------------------------------+---------+----------+--------+-------+
|NATÁLIA COELHO E SILVA|Agente Administrativo da Saúde (40h)|256697   |1994-11-25|68.0    |217    |
+----------------------+------------------------------------+---------+----------+--------+-------+



In [45]:
df2.filter('Nome like "LETICIA COELHO%"')

Nome,Cargo,Inscrição,Nascimento,Objetiva,Posição
LETICIA COELHO E ...,PROFESSOR DE EDUC...,256495,1996-05-24,52.0,567


In [8]:
df2.filter('Cargo = "AGENTE ADMINISTRATIVO DA EDUCAÇÃO"').orderBy('Posição').show(truncate=False)

+------------------------------------+------------------------------------+---------+----------+--------+-------+
|Nome                                |Cargo                               |Inscrição|Nascimento|Objetiva|Posição|
+------------------------------------+------------------------------------+---------+----------+--------+-------+
|PEDRO HENRIQUE DE MELLO RABELO      |Agente Administrativo da Saúde (40h)|257690   |1992-05-14|100.0   |1      |
|BRUNA RODRIGUES BENTO               |Agente Administrativo da Saúde (40h)|254895   |1992-07-15|100.0   |2      |
|JOAQUIM FRANCISCO TIAGO DA SILVA    |Agente Administrativo da Saúde (40h)|259033   |1987-11-11|96.0    |3      |
|FELICA ARIANY GOMES BATISTA         |Agente Administrativo da Saúde (40h)|256725   |1988-11-06|96.0    |4      |
|LUIZ FERNANDO COSTA                 |Agente Administrativo da Saúde (40h)|279225   |1994-01-05|96.0    |5      |
|ROSELENE DAS GRAÇAS AZEVEDO         |Agente Administrativo da Saúde (40h)|283932   |197

In [9]:
df2.filter('Cargo = "Agente Administrativo da Saúde (40h)"').orderBy('Posição').show(23, truncate=False)

+------------------------------------+------------------------------------+---------+----------+--------+-------+
|Nome                                |Cargo                               |Inscrição|Nascimento|Objetiva|Posição|
+------------------------------------+------------------------------------+---------+----------+--------+-------+
|PEDRO HENRIQUE DE MELLO RABELO      |Agente Administrativo da Saúde (40h)|257690   |1992-05-14|100.0   |1      |
|BRUNA RODRIGUES BENTO               |Agente Administrativo da Saúde (40h)|254895   |1992-07-15|100.0   |2      |
|JOAQUIM FRANCISCO TIAGO DA SILVA    |Agente Administrativo da Saúde (40h)|259033   |1987-11-11|96.0    |3      |
|FELICA ARIANY GOMES BATISTA         |Agente Administrativo da Saúde (40h)|256725   |1988-11-06|96.0    |4      |
|LUIZ FERNANDO COSTA                 |Agente Administrativo da Saúde (40h)|279225   |1994-01-05|96.0    |5      |
|ROSELENE DAS GRAÇAS AZEVEDO         |Agente Administrativo da Saúde (40h)|283932   |197

In [4]:
from moto.server import ThreadedMotoServer
server = ThreadedMotoServer()
server.start()

Starting a new Thread with MotoServer running on 0.0.0.0:5000...


In [5]:
import boto3
s3_endpoint_url = "http://localhost:5000"

aws_access_key_id = "mock"
aws_secret_access_key = "mock"
session = boto3.Session(aws_access_key_id=aws_access_key_id, 
              aws_secret_access_key=aws_access_key_id)
s3 = session.client('s3', endpoint_url=s3_endpoint_url)

In [6]:
s3.create_bucket(Bucket='bronze')

127.0.0.1 - - [09/Oct/2023 22:48:00] "PUT /bronze HTTP/1.1" 200 -


{'ResponseMetadata': {'RequestId': 'KmVNSOVlo9FSXrkMlboXC5FbbFPkOGxWHSGdByzMM6mrLizHHKSf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Werkzeug/2.2.3 Python/3.7.16',
   'date': 'Mon, 09 Oct 2023 22:48:00 GMT',
   'x-amzn-requestid': 'KmVNSOVlo9FSXrkMlboXC5FbbFPkOGxWHSGdByzMM6mrLizHHKSf',
   'content-type': 'text/html; charset=utf-8',
   'content-length': '160',
   'access-control-allow-origin': '*',
   'connection': 'close'},
  'RetryAttempts': 0}}

In [7]:
with open('/workspace/samples/lake/bronze/us-legislators/person.json', "rb") as f:
    s3.upload_fileobj(f, 'bronze', 'us-legislators/person.json')

127.0.0.1 - - [09/Oct/2023 22:48:00] "PUT /bronze/us-legislators/person.json HTTP/1.1" 200 -


In [9]:
spark

In [8]:
path = "s3://bronze/us-legislators/person.json"
df = spark.read.json(path)
df

23/10/09 22:48:06 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
127.0.0.1 - - [09/Oct/2023 22:48:08] "GET /bronze/?list-type=2&delimiter=%2F&max-keys=2&prefix=us-legislators%2Fperson.json%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2023 22:48:09] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2023 22:48:09] "GET /bronze/?list-type=2&delimiter=%2F&max-keys=5000&prefix=us-legislators%2Fperson.json%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2023 22:48:09] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2023 22:48:09] "GET /bronze/?list-type=2&delimiter=%2F&max-keys=2&prefix=us-legislators%2Fperson.json%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2023 22:48:09] "GET /bronze/?list-type=2&delimiter=%2F&max-keys=5000&prefix=us-legislators%2Fperson.json%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [09/O

birth_date,contact_details,death_date,family_name,gender,given_name,id,identifiers,image,images,links,name,other_names,sort_name
1944-10-15,null,null,Collins,male,Michael,0005af3a-9471-4d1...,"[{C000640, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Mac Collins,"[{bar, Mac Collin...","Collins, Michael"
1969-01-31,"[{fax, 202-226-07...",null,Huizenga,male,Bill,00aa2dc0-bfb6-441...,"[{Bill Huizenga, ...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Bill Huizenga,"[{da, Bill Huizen...","Huizenga, Bill"
1959-09-28,"[{phone, 202-225-...",null,Clawson,male,Curtis,00aca284-9323-495...,"[{C001102, biogui...",https://theunited...,[{https://theunit...,[{Wikipedia (comm...,Curt Clawson,"[{bar, Curt Claws...","Clawson, Curtis"
1930-08-14,null,2001-10-26,Solomon,male,Gerald,00b73df5-4180-441...,"[{S000675, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Gerald Solomon,"[{null, Gerald B....","Solomon, Gerald"
1960-05-28,"[{fax, 202-225-42...",null,Rigell,male,Edward,00bee44f-db04-4a7...,"[{R000589, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",E. Scott Rigell,"[{null, Scott Rig...","Rigell, Edward"
1951-05-20,"[{twitter, MikeCr...",null,Crapo,male,Michael,00f8f12d-6e27-4a2...,"[{Mike Crapo, bal...",https://theunited...,[{https://theunit...,"[{Wikipedia (da),...",Mike Crapo,"[{da, Mike Crapo,...","Crapo, Michael"
1926-05-12,null,null,Hutto,male,Earl,015d77c8-6edb-4ed...,"[{H001018, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Earl Hutto,"[{null, Earl Dewi...","Hutto, Earl"
1937-11-07,null,2015-11-19,Ertel,male,Allen,01679bc3-da21-482...,"[{E000208, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Allen Ertel,"[{null, Allen E. ...","Ertel, Allen"
1916-09-01,null,2007-11-24,Minish,male,Joseph,018247d0-2961-423...,"[{M000796, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Joseph Minish,"[{bar, Joseph Min...","Minish, Joseph"
1957-08-04,"[{phone, 202-225-...",null,Andrews,male,Robert,01b100ac-192e-4b5...,"[{A000210, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Robert E. Andrews,"[{null, Rob Andre...","Andrews, Robert"


In [10]:
df.show(vertical=True, truncate=False)

127.0.0.1 - - [12/Jul/2023 22:15:50] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:15:50] "GET /bronze/us-legislators/person.json HTTP/1.1" 206 -


-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
s3.create_bucket(Bucket='silver')

127.0.0.1 - - [12/Jul/2023 22:16:11] "PUT /silver HTTP/1.1" 200 -


{'ResponseMetadata': {'RequestId': 'WawNaI4jEV05o0jd8DCiNw96Ny0JjIAiw6Jtyy6UYAzl7mrrVfIM',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Werkzeug/2.2.3 Python/3.7.16',
   'date': 'Wed, 12 Jul 2023 22:16:11 GMT',
   'x-amzn-requestid': 'WawNaI4jEV05o0jd8DCiNw96Ny0JjIAiw6Jtyy6UYAzl7mrrVfIM',
   'content-type': 'text/html; charset=utf-8',
   'content-length': '160',
   'access-control-allow-origin': '*',
   'connection': 'close'},
  'RetryAttempts': 0}}

In [12]:
target_path = "s3://silver/legislators/"
df.write.mode("overwrite").format("delta").save(target_path)

127.0.0.1 - - [12/Jul/2023 22:17:24] "HEAD /silver/legislators/_delta_log/_last_checkpoint HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:17:24] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:17:24] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:17:24] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:17:24] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:17:24] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:17:24] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:17:24] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.

In [19]:
df.repartition(4).write.mode("overwrite").format("delta").save(target_path)

127.0.0.1 - - [12/Jul/2023 22:24:04] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:24:04] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:24:04] "GET /silver/?list-type=2&delimiter=%2F&max-keys=5000&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -


127.0.0.1 - - [12/Jul/2023 22:24:04] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:24:04] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:24:04] "GET /bronze/us-legislators/person.json HTTP/1.1" 206 -
127.0.0.1 - - [12/Jul/2023 22:24:04] "GET /bronze/us-legislators/person.json HTTP/1.1" 206 -
127.0.0.1 - - [12/Jul/2023 22:24:05] "HEAD /silver/legislators/part-00001-fe6f157f-b2e3-4a26-938e-5f466e03e965-c000.snappy.parquet HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:24:05] "HEAD /silver/legislators/part-00002-b0e7d76c-3812-4118-a048-8c24ad97d870-c000.snappy.parquet HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:24:05] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2Fpart-00001-fe6f157f-b2e3-4a26-938e-5f466e03e965-c000.snappy.parquet%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:24:05] "HEAD /silver/legislators/part-00000-9c50b543-6194-4b1f-9b25-74a9da1e4d69-c000.sn

In [18]:
response = s3.list_objects_v2(Bucket='silver')
for i in response['Contents']:
    print(i['Key'], i['Size'])

127.0.0.1 - - [12/Jul/2023 22:23:48] "GET /silver?list-type=2&encoding-type=url HTTP/1.1" 200 -


legislators/_delta_log/00000000000000000000.json 5011
legislators/_delta_log/00000000000000000001.json 2896
legislators/part-00000-1461f151-0a65-4343-955a-90ca7aaa8317-c000.snappy.parquet 649618
legislators/part-00000-7f55286d-d07e-4339-8af4-92daae77188d-c000.snappy.parquet 586895
legislators/part-00001-8f6a949f-ccc8-4f2e-96b1-afc24ae485b6-c000.snappy.parquet 582761
legislators/part-00001-fc9c26a6-e598-47a0-a4b2-71ff1e19dc2c-c000.snappy.parquet 495473


In [20]:
response = s3.list_objects_v2(Bucket='silver')
for i in response['Contents']:
    print(i['Key'], i['Size'])

127.0.0.1 - - [12/Jul/2023 22:24:14] "GET /silver?list-type=2&encoding-type=url HTTP/1.1" 200 -


legislators/_delta_log/00000000000000000000.json 5011
legislators/_delta_log/00000000000000000001.json 2896
legislators/_delta_log/00000000000000000002.json 4989
legislators/part-00000-1461f151-0a65-4343-955a-90ca7aaa8317-c000.snappy.parquet 649618
legislators/part-00000-7f55286d-d07e-4339-8af4-92daae77188d-c000.snappy.parquet 586895
legislators/part-00000-9c50b543-6194-4b1f-9b25-74a9da1e4d69-c000.snappy.parquet 286709
legislators/part-00001-8f6a949f-ccc8-4f2e-96b1-afc24ae485b6-c000.snappy.parquet 582761
legislators/part-00001-fc9c26a6-e598-47a0-a4b2-71ff1e19dc2c-c000.snappy.parquet 495473
legislators/part-00001-fe6f157f-b2e3-4a26-938e-5f466e03e965-c000.snappy.parquet 283526
legislators/part-00002-b0e7d76c-3812-4118-a048-8c24ad97d870-c000.snappy.parquet 287383
legislators/part-00003-97498dbe-6c65-4a26-9e5a-1c1f98632159-c000.snappy.parquet 290284


In [23]:
delta_table = DeltaTable.forPath(spark, target_path)

127.0.0.1 - - [12/Jul/2023 22:26:21] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:26:21] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:26:21] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:26:21] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:26:21] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:26:21] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:26:21] "GET /silver/?list-type=2&delimiter=%2F&max-keys=5000&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -


In [24]:
delta_table.history()

127.0.0.1 - - [12/Jul/2023 22:26:28] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:26:28] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:26:28] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:26:28] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:26:28] "GET /silver/?list-type=2&delimiter=%2F&max-keys=5000&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:26:28] "HEAD /silver/legislators/_delta_log HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2023 22:26:28] "GET /silver/?list-type=2&delimiter=%2F&max-keys=2&prefix=legislators%2F_delta_log%2F&fetch-owner=false HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:26:28] "GET /silver/?list-type=2&delimiter=%2F&max-keys=5000&prefix=l

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2023-07-12 22:24:07,null,null,WRITE,{mode -> Overwrit...,null,null,null,1,Serializable,false,"{numFiles -> 4, n...",null,Apache-Spark/3.3....
1,2023-07-12 22:23:42,null,null,WRITE,{mode -> Overwrit...,null,null,null,0,Serializable,false,"{numFiles -> 2, n...",null,Apache-Spark/3.3....
0,2023-07-12 22:17:33,null,null,WRITE,{mode -> Overwrit...,null,null,null,null,Serializable,false,"{numFiles -> 2, n...",null,Apache-Spark/3.3....


In [ ]:
s3.get_object()

In [26]:
df.createTempView('deltaTable')

In [28]:
df.printSchema()

root
 |-- birth_date: string (nullable = true)
 |-- contact_details: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- death_date: string (nullable = true)
 |-- family_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- given_name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- identifiers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- identifier: string (nullable = true)
 |    |    |-- scheme: string (nullable = true)
 |-- image: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: string (nullable = true)
 |-- links: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- note: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- name: string (nullable = true)

In [27]:
spark.sql("select * from deltaTable")

127.0.0.1 - - [12/Jul/2023 22:38:48] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:38:48] "GET /bronze/us-legislators/person.json HTTP/1.1" 206 -
127.0.0.1 - - [12/Jul/2023 22:38:48] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:38:48] "GET /bronze/us-legislators/person.json HTTP/1.1" 206 -


birth_date,contact_details,death_date,family_name,gender,given_name,id,identifiers,image,images,links,name,other_names,sort_name
1944-10-15,null,null,Collins,male,Michael,0005af3a-9471-4d1...,"[{C000640, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Mac Collins,"[{bar, Mac Collin...","Collins, Michael"
1969-01-31,"[{fax, 202-226-07...",null,Huizenga,male,Bill,00aa2dc0-bfb6-441...,"[{Bill Huizenga, ...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Bill Huizenga,"[{da, Bill Huizen...","Huizenga, Bill"
1959-09-28,"[{phone, 202-225-...",null,Clawson,male,Curtis,00aca284-9323-495...,"[{C001102, biogui...",https://theunited...,[{https://theunit...,[{Wikipedia (comm...,Curt Clawson,"[{bar, Curt Claws...","Clawson, Curtis"
1930-08-14,null,2001-10-26,Solomon,male,Gerald,00b73df5-4180-441...,"[{S000675, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Gerald Solomon,"[{null, Gerald B....","Solomon, Gerald"
1960-05-28,"[{fax, 202-225-42...",null,Rigell,male,Edward,00bee44f-db04-4a7...,"[{R000589, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",E. Scott Rigell,"[{null, Scott Rig...","Rigell, Edward"
1951-05-20,"[{twitter, MikeCr...",null,Crapo,male,Michael,00f8f12d-6e27-4a2...,"[{Mike Crapo, bal...",https://theunited...,[{https://theunit...,"[{Wikipedia (da),...",Mike Crapo,"[{da, Mike Crapo,...","Crapo, Michael"
1926-05-12,null,null,Hutto,male,Earl,015d77c8-6edb-4ed...,"[{H001018, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Earl Hutto,"[{null, Earl Dewi...","Hutto, Earl"
1937-11-07,null,2015-11-19,Ertel,male,Allen,01679bc3-da21-482...,"[{E000208, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Allen Ertel,"[{null, Allen E. ...","Ertel, Allen"
1916-09-01,null,2007-11-24,Minish,male,Joseph,018247d0-2961-423...,"[{M000796, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Joseph Minish,"[{bar, Joseph Min...","Minish, Joseph"
1957-08-04,"[{phone, 202-225-...",null,Andrews,male,Robert,01b100ac-192e-4b5...,"[{A000210, biogui...",https://theunited...,[{https://theunit...,"[{Wikipedia (de),...",Robert E. Andrews,"[{null, Rob Andre...","Andrews, Robert"


In [16]:
df.repartition(2).write.mode("overwrite").format("delta").save(target_path)

127.0.0.1 - - [12/Jul/2023 22:19:50] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:19:50] "HEAD /bronze/us-legislators/person.json HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 22:19:50] "GET /bronze/us-legislators/person.json HTTP/1.1" 206 -
127.0.0.1 - - [12/Jul/2023 22:19:50] "GET /bronze/us-legislators/person.json HTTP/1.1" 206 -


2

In [ ]:
server.stop()